In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import numpy as np
import re
from datetime import datetime
import nltk
import pickle
from nltk.corpus import stopwords
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer

In [ ]:
#pip install kafka-python

In [2]:
consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m.decode('ascii','ignore'))
)

In [3]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


In [4]:
producer = KafkaProducer(bootstrap_servers='localhost:9092',value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8'))
# with open('/example/data/inputfile.txt') as f:
#      lines = f.readlines()

# for line in lines:
#    producer.send('test', line)

In [5]:
consumer.subscribe('jaeger-spans')
f = open('log_prediction.csv', 'a', encoding='utf-8')
header_writer = csv.writer(f)
header_writer.writerow(["Date","Time","Duration","Comment","PredictedLabel"])
f.close()

In [6]:
def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val


In [ ]:
# for message in consumer:
#     # message value and key are raw bytes -- decode if necessary!
#     # e.g., for unicode: `message.value.decode('utf-8')`
#     # print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))
    
#     if message.value['process']['serviceName'] == 'frontend':
        
#         # open the file in the append mode
#         f1 = open('log_prediction.csv', 'a', encoding='utf-8')
#         # create the csv writer
#         writer = csv.writer(f1)
                
#         http_method = re.findall(r"{'key': 'http\.method', 'vStr': '(.+?)'}",str(message.value))
#         http_target = re.findall(r"{'key': 'http\.target', 'vStr': '(.+?)'}",str(message.value))
#         http_status_code = re.findall(r"{'key': 'http\.status_code', 'vType': '.*?', 'vInt64': '(\d+)'}",str(message.value))
#         http_url = re.findall(r"{'key': 'http\.url', 'vStr': '(.+?)'}",str(message.value))
#         duration = re.findall(r"(.+?)s",str(message.value['duration']))
        
#         if len(http_method) > 0:
#             http_method = http_method[0]
#         if len(http_target) > 0:
#             http_target = http_target[0]
#         if len(http_url) > 0:
#             http_url = http_url[0]
#         if len(http_status_code) > 0:
#             http_status_code = http_status_code[0]
#         if len(duration) > 0:
#             duration = duration[0]
        
#         print(type(http_url));
        
#         if http_method == 'POST':
#             comment = ""
#             fields = message.value['logs'][0]['fields']
#             description1 = re.findall(r"{'key': 'body', 'vStr': '(.+?)'}",str(fields))
#             if len(description1) > 0:
#                 temp = re.findall(r'"description":"(.+?)","_links":',str(description1[0]))
#                 user_id = re.findall(r'"userId":(.+?),"movieId":',str(description1[0]))
#                 movie_id = re.findall(r'"movieId":(.+?),"description":',str(description1[0]))

#                 if len(temp) > 0:
#                     comment = temp[0]
#                     user_id = user_id[0]
#                     movie_id = movie_id[0]
            
#             data = [comment,duration]
            
#             #print(data)
#             print(user_id)
#             input_val = clean_data(data[0])
#             input_val = [input_val]
#             vectorizer = pickle.load(open("vectorizer_cnn_without_duration", 'rb'))
#             input_val=vectorizer.transform(input_val).toarray()
#             input_val.shape = (1,38,38,1)
#             #print(input_val)
#             numpyData = {"array": input_val,'comment':comment,'spanId':message.value['spanId'],'userId':user_id,'movieId':movie_id}
#             producer.send('data_to_predict',numpyData)
            

In [ ]:


# # datetime object containing current date and time
# now = datetime.now()
 
# print("now =", now)

# # dd/mm/YY H:M:S
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print("date and time =", dt_string)	

In [ ]:
from datetime import datetime
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    # print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))
    
    if message.value['process']['serviceName'] == 'frontend':
        
        # open the file in the append mode
        f1 = open('log_prediction.csv', 'a', encoding='utf-8')
        # create the csv writer
        writer = csv.writer(f1)
                
        http_method = re.findall(r"{'key': 'http\.method', 'vStr': '(.+?)'}",str(message.value))
        http_target = re.findall(r"{'key': 'http\.target', 'vStr': '(.+?)'}",str(message.value))
        http_status_code = re.findall(r"{'key': 'http\.status_code', 'vType': '.*?', 'vInt64': '(\d+)'}",str(message.value))
        http_url = re.findall(r"{'key': 'http\.url', 'vStr': '(.+?)'}",str(message.value))
        duration = re.findall(r"(.+?)s",str(message.value['duration']))
        
        if len(http_method) > 0:
            http_method = http_method[0]
        if len(http_target) > 0:
            http_target = http_target[0]
        if len(http_url) > 0:
            http_url = http_url[0]
        if len(http_status_code) > 0:
            http_status_code = http_status_code[0]
        if len(duration) > 0:
            duration = duration[0]
            
            
        if http_method == 'POST':
            if "comment" in http_url:
                comment = ""
                fields = message.value['logs'][0]['fields']
                description1 = re.findall(r"{'key': 'body', 'vStr': '(.+?)'}",str(fields))
                if len(description1) > 0:
                    temp = re.findall(r'"description":"(.+?)","_links":',str(description1[0]))
                    user_id = re.findall(r'"userId":(.+?),"movieId":',str(description1[0]))
                    movie_id = re.findall(r'"movieId":(.+?),"description":',str(description1[0]))
                    
                    if len(temp) > 0:
                        comment = temp[0]
                        user_id = user_id[0]
                        movie_id = movie_id[0]
                        
                data = [comment,duration]
                print(user_id)
                input_val = clean_data(data[0])
                input_val = [input_val]
                vectorizer = pickle.load(open("vectorizer_cnn_without_duration", 'rb'))
                input_val=vectorizer.transform(input_val).toarray()
                input_val.shape = (1,38,38,1)
            #print(input_val)
                comment_date = datetime.now()
                dt_string = comment_date.strftime("%Y-%m-%d")
                numpyData = {"array": input_val,"comment_date": dt_string,'comment':comment,'spanId':message.value['spanId'],'userId':user_id,'movieId':movie_id}
                producer.send('data_to_predict',numpyData)
                
            
    
    
       

0


C:\Users\vmadmin\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0
0
0
0
0
0
0
0
0
0
0
0
